In [1]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)
from peft import PeftModel
from datasets import DatasetDict, load_dataset
from utils import set_seed, k_split
from tqdm import trange
import torch

In [2]:
task = 'sst2'
data_name = 'glue' if task in ['mnli','qnli','sst2','qqp'] else 'bigbench'
seed = 42
num_clients = 10
num_error_clients = 3
number = 5

In [3]:
model_name_or_path = 'google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
set_seed(seed)
if data_name == 'bigbench':
    dataset = load_dataset("tasksource/bigbench", task).shuffle(seed=seed)
    dataset = dataset.rename_columns({'inputs':'source','targets':'target'})
else:
    dataset = load_dataset("JsSparkYyx/NLP524", task).shuffle(seed=seed)

In [4]:
print(tokenizer)

T5TokenizerFast(name_or_path='google/flan-t5-base', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>'

In [5]:
import random
from datasets import load_dataset, Dataset, DatasetDict
def k_split_(num_clients,num_error_clients,dataset):
    data = []
    for i in range(num_clients):
        subdata = dataset.shard(num_clients,i)
        target = subdata['target']
        source = subdata['source']
        if i < num_error_clients:
            random.shuffle(target)
        subdata = Dataset.from_dict({'source':source, 'target':target})
        data.append(subdata)
    return data

train_ds = k_split_(num_clients,num_error_clients,dataset['train'])
if data_name == 'glue':
    valid_ds = k_split_(num_clients,num_error_clients,dataset['valid'])
else:
    valid_ds = k_split_(num_clients,num_error_clients,dataset['validation'])
dataset = DatasetDict({'train':train_ds[number],'valid':valid_ds[number]})
def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    model_inputs = tokenizer(examples['source'], truncation=True, max_length=None,padding=True,return_tensors='pt')
    if data_name == 'glue':
        model_inputs['labels'] = tokenizer(examples['target'], truncation=True, max_length=None,padding=True,return_tensors='pt')["input_ids"]
    else:
        model_inputs['labels'] = tokenizer([_[0] for _ in examples['target']], truncation=True, max_length=None,padding=True,return_tensors='pt')["input_ids"]
    return model_inputs
ds = (train_ds, valid_ds)

In [6]:
def retrive_data(ds,number):
    (train_ds, valid_ds) = ds
    return DatasetDict({'train':train_ds[number],'valid':valid_ds[number]})

def accuracy_score(outputs, ground_truths):
    correct = 0
    total = 0
    for output, truth in zip(outputs, ground_truths):
        if data_name == "bigbench":
            truth = truth[0]
        if output.strip().lower().replace(".", "") == truth.strip().lower().replace(".", ""):
            correct += 1
        total += 1
    return correct / total * 100

In [7]:
data = retrive_data(ds,number)

In [8]:
def evaluation(data, model, tokenizer, batch_size = 128):
    example_predictions = []
    eval_set = "valid"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.eval()
    model.to(device)
    with torch.no_grad():
        for i in trange(0, len(data[eval_set]["source"]), batch_size):
            inputs = tokenizer(
                    data[eval_set]["source"][i : i + batch_size],
                    max_length=2048,
                    return_tensors="pt",
                    padding=True,
                ).to(device)
            outputs = model.generate(
                input_ids=inputs["input_ids"], max_new_tokens=256
            )
            outputs = tokenizer.batch_decode(
                outputs.to("cpu"), skip_special_tokens=True
            )
            example_predictions.extend(outputs)

    task_perf = accuracy_score(example_predictions, data[eval_set]["target"])
    return task_perf, example_predictions

In [9]:
# model_1 = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, return_dict=True)
# lora_model = PeftModel.from_pretrained(model_1,f'JsSparkYyx/flan-t5-base-finetuned-lora-{task}-{number}')
# model_2 = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, return_dict=True)
# error_model = PeftModel.from_pretrained(model_2,f'JsSparkYyx/flan-t5-base-finetuned-lora-{task}-0')
# task_perf, example_predictions = evaluation(data,lora_model,tokenizer, batch_size=8)
# task_perf_error, example_predictions_error = evaluation(data,error_model,tokenizer, batch_size=8)
# print(f"ACC of error model: {task_perf_error}, ACC of lora model: {task_perf}")

In [10]:
from peft import get_peft_model_state_dict
lora_adaptors = []
for i in range(num_clients):
    base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, return_dict=True)
    lora_model = PeftModel.from_pretrained(base_model,f'JsSparkYyx/flan-t5-base-finetuned-lora-{task}-{i}')
    lora_adaptors.append(get_peft_model_state_dict(lora_model))

In [11]:
from adalgorithm import lorahub_aggregation
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, return_dict=True)
base_lora = PeftModel.from_pretrained(base_model,f'JsSparkYyx/flan-t5-base-finetuned-lora-{task}-{number}')
weights, lorahub_model = lorahub_aggregation(base_lora, lora_adaptors, data["valid"], tokenizer, batch_size = 5, sample_size = 5, seed = 42)


Running tokenizer on dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

> Begin to perform gradient-free optimization ...
Launching 1 jobs with new suggestions
0.0
nan
Updating fitness with value nan
39 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions


E:\Anaconda\envs\NLP\lib\site-packages\nevergrad\optimization\base.py:146: LossTooLargeWarning: Clipping very high value nan in tell (rescale the cost function?).
  warnings.warn(msg, e)
E:\Anaconda\envs\NLP\lib\site-packages\nevergrad\optimization\base.py:146: LossTooLargeWarning: Clipping very high value 5e+20 in tell (rescale the cost function?).
  warnings.warn(msg, e)


0.0012500000000000002
0.0
Updating fitness with value 0.0744208550453186
38 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
0.0025000000000000005
0.0
Updating fitness with value 0.08234742403030396
37 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
0.0025000000000000005
0.0
Updating fitness with value 0.08413163542747498
36 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
0.0025000000000000005
0.0
Updating fitness with value 0.06285675883293153
35 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
0.00375
0.0
Updating fitness with value 0.057635598182678216
34 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
0.005000000000000001
0.0
Updating fitness with value 0.03943074434995651
33 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
0.00625
0.0
Updating fitness with value 2.0876838684082033
32 remaining budget and 0 running jobs
Launching 1

In [12]:
from adalgorithm import average_aggregation
from utils import evaluation
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, return_dict=True)
base_lora = PeftModel.from_pretrained(base_model,f'JsSparkYyx/flan-t5-base-finetuned-lora-{task}-{number}')
avg_model = average_aggregation(base_lora,lora_adaptors)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, return_dict=True)
base_lora = PeftModel.from_pretrained(base_model,f'JsSparkYyx/flan-t5-base-finetuned-lora-{task}-{number}')
no_noise_model = average_aggregation(base_lora,lora_adaptors[3:])
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, return_dict=True)
client_model = PeftModel.from_pretrained(base_model,f'JsSparkYyx/flan-t5-base-finetuned-lora-{task}-{number}')
# task_perf_avg, example_predictions = evaluation(data,avg_model,tokenizer, task, batch_size=8)
# task_perf_client, example_predictions_error = evaluation(data,client_model,tokenizer, task, batch_size=8)
# task_perf_lorahub, example_predictions_error = evaluation(data,lorahub_model,tokenizer, task, batch_size=8)
# task_perf_no_noise, example_predictions_error = evaluation(data,no_noise_model,tokenizer, task, batch_size=8)
task_perf_avg, example_predictions = evaluation(data,avg_model,tokenizer, batch_size=8)
task_perf_client, example_predictions_error = evaluation(data,client_model,tokenizer, batch_size=8)
task_perf_lorahub, example_predictions_error = evaluation(data,lorahub_model,tokenizer, batch_size=8)
task_perf_no_noise, example_predictions_error = evaluation(data,no_noise_model,tokenizer, batch_size=8)
print(f"ACC of client's model: {task_perf_client}, ACC of average aggregated model: {task_perf_avg}, ACC of lorahub model: {task_perf_lorahub}, ACC of no noise model: {task_perf_no_noise}")

  0%|                                                                                                                                                                                 | 0/11 [00:00<?, ?it/s]E:\Anaconda\envs\NLP\lib\site-packages\transformers\tokenization_utils_base.py:2411: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:04<00:00,  2.58it/s]

ACC of client's model: 94.25287356321839, ACC of average aggregated model: 90.80459770114942, ACC of lorahub model: 95.40229885057471, ACC of no noise model: 91.95402298850574


In [13]:
import numpy as np
a = np.zeros((5,5))
a[1] = [1,2,3,4,5]
print(a)

[[0. 0. 0. 0. 0.]
 [1. 2. 3. 4. 5.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [14]:
print(type(task_perf_client))

<class 'float'>


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # 定义10个可学习的参数
        self.weights = nn.Parameter(torch.rand(10))

    def forward(self, x):
        # 将每个参数乘以输入矩阵的对应列，然后对列求和，得到10x1的向量
        return torch.matmul(x, self.weights.view(-1, 1))

model = Model()
optimizer = optim.Adam([model.weights], lr=0.01)

In [16]:
training_steps = 200
for i in tqdm(range(training_steps)):
    optimizer.zero_grad()
    aggregated_weights = model(weights)
    ########
    # 合并成一个lora
    # loss = evaludation
    weighted_model = weighted_aggregation(base_lora, lora_adaptors, aggregated_weights)
    task_perf, exmaple_predictions_error = evaluation(data, weighted_model, tokenizer, task, batch_size=8)
    task_perf = 

    ########
    loss.backward()
    optimizer.step()
    if i % 10 == 0:
        print(loss.item())

SyntaxError: invalid syntax (1547965237.py, line 10)